In [1]:
import numpy as np
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix
import torch
from torch.autograd import Variable
import pandas as pd
import copy

In [2]:
EPOCH = 50
BATCH_SIZE = 1000
LR = 3e-3
n_factor = 4
use_gpu = torch.cuda.is_available()

In [9]:
trains = ['ml-10M100K/r1.train','ml-10M100K/r2.train','ml-10M100K/r3.train','ml-10M100K/r4.train','ml-10M100K/r5.train']
tests = ['ml-10M100K/r1.test','ml-10M100K/r2.test','ml-10M100K/r3.test','ml-10M100K/r4.test','ml-10M100K/r5.test']

In [10]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df_trains =[pd.read_csv(t, sep='::', names=names,engine='python') for t in trains]
df_tests = [pd.read_csv(t, sep='::', names=names,engine='python') for t in tests]

In [11]:
def get_movielens_ratings(df):
    n_users = max(df.user_id.unique())
    n_items = max(df.item_id.unique())

    interactions = lil_matrix( (n_users,n_items), dtype=float) #np.zeros((n_users, n_items))
    for row in df.itertuples():
        interactions[row[1] - 1, row[2] - 1] = row[3]
    return interactions

In [16]:
ratings_arr = [get_movielens_ratings(df) for df in df_trains]
ratings_arr_test = [get_movielens_ratings(df) for df in df_tests]

In [24]:
def get_batch(batch_size,ratings):
    # Sort our data and scramble it
    rows, cols = ratings.shape
    p = np.random.permutation(rows)
    
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = p[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch
        
def get_batch_in_seqence(batch_size, ratings):
    rows, cols = ratings.shape
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = range(sindex,eindex)
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch

In [25]:
def run_epoch(model, batch_size, ratings, loss_func, reg_loss_func):
    epoch_ave_loss = 0.0
    batch_cnt = 0
    for i,batch in enumerate(get_batch(batch_size, ratings)):
        # Set gradients to zero
        reg_loss_func.zero_grad()
        
        # Turn data into variables
        interactions = Variable(torch.FloatTensor(ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(ratings.shape[1])))
        if use_gpu:
            interactions = interactions.cuda()
            rows = rows.cuda()
            cols = cols.cuda()
    
        # Predict and calculate loss
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        epoch_ave_loss += loss.data[0]
        batch_cnt += 1
        # Backpropagate
        loss.backward()
    
        # Update the parameters
        reg_loss_func.step()
        
    return model, epoch_ave_loss/batch_cnt

def run_test_batchly(model, ratings, loss_func, batch_size=1000):
    for i,batch in enumerate(get_batch_in_seqence(batch_size, ratings)):
        # Turn data into variables
        interactions = Variable(torch.FloatTensor(ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(ratings.shape[1])))
        if use_gpu:
            interactions = interactions.cuda()
            rows = rows.cuda()
            cols = cols.cuda()
        # Predict and calculate loss
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        yield predictions, loss.data[0]

In [26]:
def train_model(model, ratings, weight_decay, num_epochs=25, batch_size=1000):
    reg_loss_func = torch.optim.SGD(model.parameters(), lr = 3e-3, weight_decay = weight_decay)
    loss_func = torch.nn.MSELoss()
    best_model_wts = copy.deepcopy(model.state_dict())
    
    lowest_loss = 1e10
    print('Epoch\tAve-loss')
    for epoch in range(num_epochs):
        model, ave_loss = run_epoch(model, batch_size, ratings, loss_func, reg_loss_func)
        print('{}\t{}'.format(epoch, ave_loss))
        if lowest_loss > ave_loss:
            lowest_loss = ave_loss
            best_model_wts = copy.deepcopy(model.state_dict())
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
class MatrixFactorization(torch.nn.Module):    
    def __init__(self, n_users, n_items, n_factors=4):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=False)
        if use_gpu:
            self.user_factors = self.user_factors.cuda()
            self.item_factors = self.item_factors.cuda()
        # Also should consider fitting overall bias (self.mu term) and both user and item bias vectors
        # Mu is 1x1, user_bias is 1xn_users. item_bias is 1xn_items
    
    # For convenience when we want to predict a sinble user-item pair. 
    def predict(self, user, item):
        # Need to fit bias factors
        return (pred + self.user_factors(user) * self.item_factors(item)).sum(1)
    
    # Much more efficient batch operator. This should be used for training purposes
    def forward(self, users, items):
        # Need to fit bias factors
        return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))

In [20]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=4):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=False)
        self.item_biases = torch.nn.Embedding(n_items, 1, sparse=False)
        if use_gpu:
            self.user_factors = self.user_factors.cuda()
            self.item_factors = self.item_factors.cuda()
            self.item_biases = self.item_biases.cuda()
        
    def forward(self, users, items):
        constant_user_biases = Variable(torch.FloatTensor(np.transpose([np.ones(len(users))])))
        if use_gpu:
            constant_user_biases = constant_user_biases.cuda()
        biases = torch.mm(constant_user_biases, torch.transpose(self.item_biases(items),0,1))
        linear = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        return biases + linear

In [15]:
models_without_biases = {}
for i in range(5):
    ratings = ratings_arr[i]
    user_num, item_num = ratings.shape
    for weight_decay in [0.001, 0.01, 0.1]:
        print('trainset:{}, weight_decay:{}'.format(i, weight_decay))
        model = MatrixFactorization(user_num, item_num, 4)
        if use_gpu:
            model = model.cuda()
        model = train_model(model, ratings, weight_decay, EPOCH)
        model_name = str(i)+ '_' + str(weight_decay)
        models_without_biases[model_name] = model

trainset:0, weight_decay:0.001
Epoch	Ave-loss
0	4.014335456821653
1	4.013113492065006
2	4.007376299964057
3	4.006175494856304
4	3.9995601375897727
5	3.998066249820921
6	3.9943118029170566
7	3.9897547231780157
8	3.9877712263001337
9	3.9834243191613092
10	3.9808948834737143
11	3.9740902649031744
12	3.97379493382242
13	3.969550155931049
14	3.966994331942664
15	3.9635450177722507
16	3.9581759638256497
17	3.9558136694961124
18	3.9526692661974163
19	3.9469648467169867
20	3.9446189370420246
21	3.9429165754053326
22	3.9367858204576702
23	3.9338105387157865
24	3.9327875640657215
25	3.9262405269675784
26	3.922848857111401
27	3.921896424558428
28	3.9169916411240897
29	3.9146319097942777
30	3.909904337591595
31	3.906446543004778
32	3.9030089808834925
33	3.8988591167661877
34	3.8971494999196796
35	3.892407202058368
36	3.8900917801592083
37	3.88582307100296
38	3.880741665760676
39	3.8800672226481967
40	3.875399592849943
41	3.873704820871353
42	3.8691502842638226
43	3.8662931554847293
44	3.8616203831

In [ ]:
models_with_biases = {}
for i in range(5):
    ratings = ratings_arr[i]
    user_num, item_num = ratings.shape
    for weight_decay in [0.1, 0.01, 0.001]:
        print('trainset:{}, weight_decay:{}'.format(i, weight_decay)) 
        model = BiasedMatrixFactorization(user_num, item_num, 4)
        if use_gpu:
            model = model.cuda()
        model = train_model(model, ratings, weight_decay, EPOCH)
        model_name = str(i)+ '_' + str(weight_decay)
        models_with_biases[model_name] = model

trainset:0, weight_decay:0.1
Epoch	Ave-loss
0	4.059445771906111
1	3.6979528963565826
2	3.3670172227753534
3	3.0732666320270963
4	2.8104589647716947
5	2.5801185535060034
6	2.3781093491448297
7	2.2042880058288574
8	2.0415328989426293
9	1.8652843882640202
10	1.7000817822085486
11	1.5489957862430148
12	1.419359490275383
13	1.3030792209837172
14	1.2018069856696658
15	1.113113209605217
16	1.0334666222333908
17	0.9463949427008629
18	0.8626923668715689
19	0.7884290582603879
20	0.7226263135671616
21	0.6630616585413615
22	0.611938490635819
23	0.5666840531759791
24	0.5277889515790675
25	0.48597100501259166
26	0.444123783459266
27	0.40679725093974006
28	0.3728519131739934
29	0.3431900545126862
30	0.3168380219075415
31	0.29377154509226483
32	0.2737815073794789
33	0.2550428097860681
34	0.2343995649781492
35	0.21525708730849955
36	0.1983705283039146
37	0.18298570811748505
38	0.1695170599139399
39	0.15746413088507122
40	0.14687482433186638


In [69]:
for item in models_with_biases.items():
    torch.save(item[1],'model/with_biases_'+str(item[0]))
    
for item in models_without_biases.items():
    torch.save(item[1],'model/without_biases_'+str(item[0]))

C:\Users\lm769\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type BiasedMatrixFactorization. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\lm769\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type MatrixFactorization. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:

rst = open('assign5_r5results.tsv','w')
u_ids_in_test = set(df_tests[4].user_id.unique())
user_id = 1
for pred, loss in run_test_batchly(bias_model, ratings_arr_test[4], torch.nn.MSELoss()):
    for user in pred:
        if base_id %10000 == 0: 
            print(base_id)
        tops = torch.topk(user,50)[1].data.cpu().numpy()
        recs = np.setdiff1d(tops, np.array(ratings_arr_test[4].rows[user_id-1]), assume_unique=True)
        if len(recs) < 5:
            print('ERROR!!!\t', base_id)
        if user_id in u_ids_in_test:
            recs = recs[:5]
            rst.write(str(user_id) + '\t' + '\t'.join([str(rec) for rec in recs]) + '\n')
        base_id += 1
rst.close()      

AttributeError: module 'torch._C' has no attribute '_cuda_getDevice'